In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os

nodr_dir = "/content/drive/MyDrive/EECE 490 Project/All_NoDR_Images_Processed_Balanced"
dr_dir = "/content/drive/MyDrive/EECE 490 Project/All_DR_Images_Processed_Balanced"

def count_images(directory):
    image_extensions = {".png"}
    return sum(1 for file in os.listdir(directory) if os.path.splitext(file)[1].lower() in image_extensions)

nodr_count = count_images(nodr_dir)
dr_count = count_images(dr_dir)

print(f"No_DR images: {nodr_count}")
print(f"DR images: {dr_count}")


In [ ]:
import os
from PIL import Image

# === Directories ===
nodr_dir = "/content/drive/MyDrive/EECE 490 Project/All_NoDR_Images_Processed_Balanced"
dr_dir = "/content/drive/MyDrive/EECE 490 Project/All_DR_Images_Processed_Balanced"
output_nodr = "/content/drive/MyDrive/EECE 490 Project/All_NoDR_Images_Resized"
output_dr = "/content/drive/MyDrive/EECE 490 Project/All_DR_Images_Resized"

os.makedirs(output_nodr, exist_ok=True)
os.makedirs(output_dr, exist_ok=True)

# === Function for Resizing ===
def resize_image(image_path, save_path):
    img = Image.open(image_path).convert("RGB")
    resized = img.resize((256, 256), resample=Image.BICUBIC)
    resized.save(save_path)

# === Apply to both folders ===
def process_folder(input_dir, output_dir):
    for img_name in os.listdir(input_dir):
        if img_name.lower().endswith(".png"):
            input_path = os.path.join(input_dir, img_name)
            output_path = os.path.join(output_dir, img_name)
            resize_image(input_path, output_path)

process_folder(nodr_dir, output_nodr)
process_folder(dr_dir, output_dr)

print("Resizing complete! Images saved.")


In [ ]:
import os
import shutil
import random

# === Input Directories ===
nodr_dir = "/content/drive/MyDrive/EECE 490 Project/All_NoDR_Images_Resized"
dr_dir = "/content/drive/MyDrive/EECE 490 Project/All_DR_Images_Resized"

# === Output Base Directory ===
output_dir = "/content/drive/MyDrive/EECE 490 Project/RETFound_Split"

# === Split Ratios ===
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2

# === Function to Split Data ===
def split_and_copy(src_dir, dst_dir, class_name):
    images = [img for img in os.listdir(src_dir) if img.lower().endswith(".png")]
    random.shuffle(images)

    total = len(images)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)

    splits = {
        "train": images[:train_end],
        "val": images[train_end:val_end],
        "test": images[val_end:]
    }

    for split, split_images in splits.items():
        split_dir = os.path.join(dst_dir, split, class_name)
        os.makedirs(split_dir, exist_ok=True)
        for img in split_images:
            shutil.copy(os.path.join(src_dir, img), os.path.join(split_dir, img))

# === Perform Splits ===
split_and_copy(nodr_dir, output_dir, "no_dr")
split_and_copy(dr_dir, output_dir, "dr")

print("Dataset split complete!")


In [ ]:
import os

# === Directory to Check ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RETFound_Split"
image_extensions = {".png"}

# === Count Images per Subfolder ===
def count_images_per_folder(base_path):
    for split in ["train", "val", "test"]:
        print(f"\n📂 {split.upper()} SET")
        split_path = os.path.join(base_path, split)
        if not os.path.exists(split_path):
            print(f"  🚫 Path does not exist: {split_path}")
            continue

        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if os.path.isdir(class_path):
                image_count = sum(
                    1 for file in os.listdir(class_path)
                    if os.path.splitext(file)[1].lower() in image_extensions
                )
                print(f"  - {class_name}: {image_count} images")

# === Run It ===
count_images_per_folder(base_dir)


In [ ]:
import os
from PIL import Image
from torchvision import transforms
import random

# === Directories ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RETFound_Split"
output_dir = "/content/drive/MyDrive/EECE 490 Project/RETFound_Split_Augmented"
os.makedirs(output_dir, exist_ok=True)

# === RETFound Augmentations ===
augment = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.2, 1.0)),  # 20%-100% cropping
    transforms.RandomHorizontalFlip(),
])

# === Function to Apply Augmentations and Save ===
def augment_and_save(split):
    split_dir = os.path.join(base_dir, split)
    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        output_class_dir = os.path.join(output_dir, split, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_name)
                img = Image.open(img_path).convert("RGB")

                # Apply augmentations
                aug_img = augment(img)

                # Save augmented image (append '_aug' to filename)
                output_img_path = os.path.join(output_class_dir, img_name.split('.')[0] + '_aug.png')
                aug_img.save(output_img_path)

# === Run for All Splits ===
for split in ["train", "val", "test"]:
    augment_and_save(split)

print("Augmentation complete! Augmented images saved.")


In [ ]:
import os
from PIL import Image

# === Directory to Check ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RETFound_Split_Augmented"

# === Track wrong-sized images ===
wrong_sized_images = []

# === Function to Check Image Sizes ===
def check_image_sizes(split):
    split_dir = os.path.join(base_dir, split)
    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_name)
                try:
                    img = Image.open(img_path)
                    if img.size != (224, 224):
                        wrong_sized_images.append((img_path, img.size))
                except Exception as e:
                    print(f"🚨 Error reading {img_path}: {e}")

# === Check All Splits ===
for split in ["train", "val", "test"]:
    check_image_sizes(split)

# === Summary ===
if wrong_sized_images:
    print("\n🚨 Found images with incorrect sizes:")
    for path, size in wrong_sized_images:
        print(f"{path} - Size: {size}")
else:
    print("✅ All images are 224×224!")


In [ ]:
import os

# === Directory to Check ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RETFound_Split_Augmented"

# === Image extensions ===
image_extensions = {".png", ".jpg", ".jpeg"}

# === Function to Count Images ===
def count_images(split):
    print(f"\n📂 {split.upper()} SET")
    split_dir = os.path.join(base_dir, split)
    total = 0
    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        count = sum(
            1 for img in os.listdir(class_dir)
            if os.path.splitext(img)[1].lower() in image_extensions
        )
        total += count
        print(f"  - {class_name}: {count} images")
    print(f"  ➡️ TOTAL: {total} images")

# === Count for All Splits ===
for split in ["train", "val", "test"]:
    count_images(split)
